In [ ]:
from importlib import reload
import geopandas as gpd

from scripts import preprocessing as pre

from config import Preprocessing
config = reload(Preprocessing).Configuration()

In [ ]:
# Read the training area and training polygons
trainingArea = gpd.read_file(config.training_base_dir/config.training_area_fn)
trainingPolygon = gpd.read_file(config.training_base_dir/config.training_polygon_fn)

print(
    f"Read a total of {trainingPolygon.shape[0]} object polygons and "
    "{trainingArea.shape[0]} training areas."
)
print(f"Polygons will be assigned to training areas in the next steps.")

In [ ]:
# Check if the training areas and the training polygons have the same crs
if trainingArea.crs != trainingPolygon.crs:
    print("Training area CRS does not match training_polygon CRS")
    # Areas are less in number so conversion should be faster
    targetCRS = trainingPolygon.crs  
    trainingArea = trainingArea.to_crs(targetCRS)
    
print(trainingPolygon.crs)
print(trainingArea.crs)
assert trainingPolygon.crs == trainingArea.crs

In [ ]:
# Assign serial IDs to training areas and polygons
trainingArea["id"] = range(trainingArea.shape[0])
trainingPolygon["id"] = range(trainingPolygon.shape[0])

In [ ]:
# areasWithPolygons contains the object polygons and weighted boundaries for each area!
areas_with_polygons = pre.dividePolygonsInTrainingAreas(trainingPolygon, trainingArea)

In [ ]:
config = reload(Preprocessing).Configuration()
input_images = pre.read_input_images(
    config.raw_image_base_dir,
    config.raw_image_file_type,
    config.raw_image_suffix,
)
print(f"Found a total of {len(input_images)} pair of raw image(s) to process!")

In [ ]:
pre = reload(pre)
config = reload(Preprocessing).Configuration()

In [ ]:
# Run the main function for extracting part of ndvi and 
# pan images that overlap with training areas
pre.findOverlap(
    input_images,
    areas_with_polygons,
    config.path_to_write,
    config.extracted_bands_folder,
    config.extracted_annotation_folder,
    config.extracted_boundary_folder,
    config.bands,
)

In [ ]:
# Display extracted image
sampleImage = "_0.png"
fn = config.path_to_write/config.extracted_ndvi_filename/sampleImage

ndvi_img = Image.open(fn)
pan_img = Image.open(
    fn.replace(config.extracted_ndvi_filename, config.extracted_pan_filename)
)
read_ndvi_img = np.array(ndvi_img)
read_pan_img = np.array(pan_img)
annotation_im = Image.open(
    fn.replace(config.extracted_ndvi_filename, config.extracted_annotation_filename)
)
read_annotation = np.array(annotation_im)
weight_im = Image.open(
    fn.replace(config.extracted_ndvi_filename, config.extracted_boundary_filename)
)
read_weight = np.array(weight_im)
all_images = np.array([read_ndvi_img, read_pan_img, read_annotation, read_weight])

display_images(np.expand_dims(np.transpose(all_images, axes=(1, 2, 0)), axis=0))
# plt.imshow(read_weight)